<a target="_blank" href="https://colab.research.google.com/github/weaviate/recipes/blob/main/weaviate-features/similarity-search/similarity_search_transformers.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

`text2vec-transformers` is **only** available through Weaviate open-source. Here are options to select your desired model: 

1. [Pre-built transformers model containers](https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-transformers#pre-built-images)

2. [Any model from Hugging Face Model Hub](https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-transformers#option-2-use-any-publicly-available-hugging-face-model)

3. [Use any private or local PyTorch or Tensorflow transformer model](https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-transformers#option-3-custom-build-with-a-private-or-local-model)

## Dependencies

In [ ]:
!pip install weaviate-client

## Connect to Weaviate

In [ ]:
import weaviate
from weaviate.config import AdditionalConfig

# Connect to your local Weaviate instance deployed with Docker
client = weaviate.connect_to_local(
    additional_config=AdditionalConfig(timeout=[900,1200])
)

client.is_ready()

## Create a collection
> Collection stores your data and vector embeddings.

In [30]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "JeopardyQuestion", and then you need to re-import it again.
import weaviate.classes.config as wc

# Delete the collection if it already exists
if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyQuestion",

    vectorizer_config=wc.Configure.Vectorizer.text2vec_transformers( # specify the vectorizer and model type you're using
        # pooling_strategy="masked_mean"
    ),

    properties=[ # defining properties (data schema) is optional
        wc.Property(name="Question", data_type=wc.DataType.TEXT), 
        wc.Property(name="Answer", data_type=wc.DataType.TEXT),
        wc.Property(name="Category", data_type=wc.DataType.TEXT, skip_vectorization=True), 
    ]
)

print("Successfully created collection: JeopardyQuestion.")

Successfully created collection: JeopardyQuestion.


## Import the Data

In [ ]:
import requests, json
url = 'https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text)

# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

# Insert data objects
with jeopardy.batch.fixed_size(
    batch_size=2,
    # concurrent_requests=1
) as batch:
    for item in data:
        print("Adding", item)
        batch.add_object(item)

print("Import Complete")

if (len(jeopardy.batch.failed_objects) > 0):
    print("There were some errors")
    for fo in jeopardy.batch.failed_objects:
        print(fo)

In [ ]:
jeopardy = client.collections.get("JeopardyQuestion")
print(jeopardy.aggregate.over_all())

## Query Weaviate: Similarity Search (Text objects)

Similarity search options for text objects in **Weaviate**:

1. [near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

2. [near_object](https://weaviate.io/developers/weaviate/search/similarity#an-object)

3. [near_vector](https://weaviate.io/developers/weaviate/search/similarity#a-vector)

### nearText Example

Find a `JeopardyQuestion` about "animals in movies". Limit it to only 4 responses.

In [ ]:
# note, you can reuse the collection object from the previous cell.
# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

response = jeopardy.query.near_text(
    query="african beasts",
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

Return vector embeddings.

In [ ]:
response = jeopardy.query.near_text(
    query="african beasts",
    include_vector=True,
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2))
    print("Vector:", item.vector, "\n")

Now, also request the `distance` for each returned item.

In [ ]:
import weaviate.classes.query as wq

response = jeopardy.query.near_text(
    query="african beasts",
    return_metadata=wq.MetadataQuery(distance=True),
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Distance:", item.metadata.distance)
    print("Data:", item.properties, "\n")

### nearObject Example

Search through the `JeopardyQuestion` class to find the top 4 objects closest to id `a1dd67f9-bfa7-45e1-b45e-26eb8c52e9a6`. (The id was taken from the query above)

In [ ]:
response = jeopardy.query.near_object(
    near_object="a1dd67f9-bfa7-45e1-b45e-26eb8c52e9a6", # replace with your id of interest
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", item.properties, "\n")

### nearVector Example

Search through the `JeopardyQuestion` class to find the top 2 objects closest to the query vector `[-0.0125526935, -0.021168863, ... ]`

In [ ]:
response = jeopardy.query.near_vector(
    near_vector=[-0.0125526935, -0.021168863, ... ], # your vector object goes here
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", item.properties, "\n")